**Корректность проверена на Python 3.6:**
+ gensim 3.4.0

# Пример использования библиотеки gensim для тематического моделирования

Такая полезная теорема Байеса! :)

![comic1](http://imgs.xkcd.com/comics/seashell.png)

In [2]:
from gensim import corpora, models

## Загрузка данных 

### Формат UCI Bag of Words 

Из коллекции текстов удаляются лишние символы и проводится лемматизация (приведение к нормальной форме). Далее создаются словари слов и их частот в документах.

#### Список слов

In [11]:
print(*open("data/vocab.xkcd.txt").readlines()[:4], sep="")

boy
sits
barrel
floating



#### Частоты 

In [10]:
print(*open("data/docword.xkcd.txt").readlines()[:4], sep="")

1265
14352
87409
1 1 2



<pre>
Количество документов
Количество уникальных слов
Общее количество слов
Номер документа - Номер слова - Частота
...
</pre>

### Загрузка 

In [12]:
# Импортируем данные в формте UCI Bag of Words
data = corpora.UciCorpus("data/docword.xkcd.txt", "data/vocab.xkcd.txt")
dictionary = data.create_dictionary()

## Модель 

In [18]:
# обучение модель
ldamodel = models.ldamodel.LdaModel(corpus=data, 
                                  id2word=dictionary, 
                                  num_topics=5, 
                                  passes=20, #количество проходов
                                    # параметры распр. Дирихле, числа-вектор
                                  alpha=1.25, 
                                  eta=1.25
                                   )

In [19]:
! mkdir models

In [20]:
# Сохранение модели
ldamodel.save("models/ldamodel_xkcd")

In [21]:
# Загрузка модели
ldamodel = models.ldamodel.LdaModel.load("models/ldamodel_xkcd")

## Метрики качества 

In [22]:
# выводим топы слов
for t, top_words in ldamodel.print_topics(num_topics=10, 
                                          num_words=10):
    print("Topic", t, ":", top_words)

Topic 0 : 0.016*"b'guy'" + 0.011*"b'girl'" + 0.007*"b'text'" + 0.006*"b'title'" + 0.005*"b'hat'" + 0.004*"b'one'" + 0.004*"b'man'" + 0.004*"b'just'" + 0.004*"b'figure'" + 0.003*"b'like'"
Topic 1 : 0.035*"b'man'" + 0.014*"b'woman'" + 0.011*"b'text'" + 0.009*"b'title'" + 0.005*"b'computer'" + 0.005*"b'one'" + 0.004*"b'just'" + 0.004*"b'hat'" + 0.003*"b'two'" + 0.003*"b'can'"
Topic 2 : 0.024*"b'person'" + 0.006*"b'text'" + 0.005*"b'title'" + 0.004*"b'one'" + 0.003*"b'people'" + 0.003*"b'line'" + 0.003*"b'two'" + 0.003*"b'panel'" + 0.003*"b'time'" + 0.002*"b'student'"
Topic 3 : 0.002*"b'map'" + 0.002*"b'labeled'" + 0.002*"b'island'" + 0.001*"b'mark'" + 0.001*"b'chart'" + 0.001*"b'text'" + 0.001*"b'title'" + 0.001*"b'han'" + 0.001*"b'north'" + 0.001*"b'line'"
Topic 4 : 0.002*"b'day'" + 0.001*"b'turtle'" + 0.001*"b'spirit'" + 0.001*"b'lake'" + 0.001*"b'reporter'" + 0.001*"b'egg'" + 0.001*"b'radio'" + 0.001*"b'thee'" + 0.001*"b'summers'" + 0.001*"b'jewel'"


In [27]:
# Вычисляем логарифм перплексии и немного преобразуем, чтобы привести к общепринятому виду
perplexity = ldamodel.log_perplexity(list(data))
print(2**(-perplexity))

368.5484096185736


In [28]:
perp = ldamodel.bound(data)
2**(-perp/float(87409))

368.5484146411468

Перплексия должна измерятся на одном и том же объеме данных и числа тем. Её можно использовать только для сравнения моделей.

In [29]:
# Добавление в модель новых документов (дообучение), содержащихся в новом корупсе data2
ldamodel.update(data2, passes=10)

NameError: name 'data2' is not defined

In [30]:
# Получение распределения тем для конкретного документа
doc = list(data)[0]
ldamodel.get_document_topics(doc)

[(0, 0.6622993),
 (1, 0.13572086),
 (2, 0.08688501),
 (3, 0.056953747),
 (4, 0.058141112)]

Эти люди не знают про тематические модели:

![comic2](http://imgs.xkcd.com/comics/the_problem_with_wikipedia.png) | ![comic3](http://imgs.xkcd.com/comics/mystery_news.png)